#VADER

In [1]:
pip install emoji


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 9.1 MB/s eta 0:00:00


In [3]:


import pandas as pd
import re
import emoji
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# Unduh resource yang dibutuhkan NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('punkt_tab') # Download the missing 'punkt_tab' resource

# Muat data
data = pd.read_excel('/content/Ntweets.xlsx')  # Sesuaikan path dengan lokasi file Anda
data = data.rename(columns={'date': 'Date', 'renderedContent': 'Tweet'})

# Konversi kolom 'Date' menjadi format tanggal
data['Date'] = pd.to_datetime(data['Date']).dt.date

# Fungsi untuk pra-pemrosesan tweet
def preprocess_tweet(tweet):
    new_tweet = tweet
    new_tweet = re.sub(r'https?://[^ ]+', '', new_tweet)  # Hapus URL
    new_tweet = re.sub(r'@[^ ]+', '', new_tweet)  # Hapus mentions
    new_tweet = re.sub(r'#', '', new_tweet)  # Hapus simbol '#'
    new_tweet = re.sub(r'([A-Za-z])\1{2,}', r'\1', new_tweet)  # Mengurangi pengulangan karakter
    new_tweet = emoji.demojize(new_tweet)  # Ubah emoji menjadi teks
    new_tweet = re.sub(r'[^A-Za-z ]', '', new_tweet)  # Hapus karakter non-alphabet
    new_tweet = new_tweet.lower()  # Konversi ke huruf kecil

    # Tokenisasi, hapus stopwords, dan stem
    tokens = word_tokenize(new_tweet)
    porter = PorterStemmer()
    tokens = [porter.stem(token) for token in tokens if token not in stopwords.words('english')]

    return ' '.join(tokens)

# Terapkan pra-pemrosesan ke kolom 'Tweet'
data['Processed_Tweet'] = data['Tweet'].apply(preprocess_tweet)

# Inisialisasi Sentiment Intensity Analyzer
sentiment_analyzer = SentimentIntensityAnalyzer()

# Analisis sentimen dan tambahkan kolom untuk skor sentimen
data['Negative'] = 0.0
data['Neutral'] = 0.0
data['Positive'] = 0.0
data['Compound'] = 0.0

for indx, row in data.iterrows():
    sentence_sentiment = sentiment_analyzer.polarity_scores(row['Processed_Tweet'])
    data.at[indx, 'Negative'] = sentence_sentiment['neg']
    data.at[indx, 'Neutral'] = sentence_sentiment['neu']
    data.at[indx, 'Positive'] = sentence_sentiment['pos']
    data.at[indx, 'Compound'] = sentence_sentiment['compound']

# Salin dataframe dan tambahkan label sentimen
labeled_data = data.copy()
for indx, row in labeled_data.iterrows():
    if row['Compound'] > 0.5:
        labeled_data.at[indx, 'Sentiment'] = 'Positive'
    elif row['Compound'] < -0.5:
        labeled_data.at[indx, 'Sentiment'] = 'Negative'
    else:
        labeled_data.at[indx, 'Sentiment'] = 'Neutral'

# Simpan data yang telah diproses ke file CSV
labeled_data.to_csv('tweet.csv', index=False)

# Tampilkan beberapa baris pertama dari hasil akhir
print(labeled_data[['Date', 'Tweet', 'Processed_Tweet', 'Negative', 'Neutral', 'Positive', 'Compound', 'Sentiment']].head())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


         Date                                              Tweet  \
0  2022-07-11  Important since $nflx may start cracking down ...   
1  2022-07-11  @QTRResearch $GOOGL technicals look good (brok...   
2  2022-07-11  @MikeInTheClutch @joecarlsonshow Exactly. Ther...   
3  2022-07-11  And that's how You negotiate an exclusivity de...   
4  2022-07-11  Trending Stocks on #fintwit: \n\nSource: finap...   

                                     Processed_Tweet  Negative  Neutral  \
0  import sinc nflx may start crack password shar...     0.000    0.804   
1  googl technic look good broke day split cataly...     0.088    0.820   
2  exactli may never way weigh babatrademark valu...     0.095    0.905   
3  that negoti exclus dealthat nflx need key show...     0.000    0.804   
4  trend stock fintwit sourc finapsel tsla twtr s...     0.000    1.000   

   Positive  Compound Sentiment  
0     0.196    0.2960   Neutral  
1     0.091    0.0258   Neutral  
2     0.000   -0.2732   Neutral  
3   

In [4]:
# Simpan data yang telah diproses ke file CSV
labeled_data.to_csv('tweet.csv', index=False)

# Simpan data yang telah diproses ke file Excel (XLSX)
labeled_data.to_excel('tweet.xlsx', index=False)

# Tampilkan beberapa baris pertama dari hasil akhir
print(labeled_data[['Date', 'Tweet', 'Processed_Tweet', 'Negative', 'Neutral', 'Positive', 'Compound', 'Sentiment']].head())


         Date                                              Tweet  \
0  2022-07-11  Important since $nflx may start cracking down ...   
1  2022-07-11  @QTRResearch $GOOGL technicals look good (brok...   
2  2022-07-11  @MikeInTheClutch @joecarlsonshow Exactly. Ther...   
3  2022-07-11  And that's how You negotiate an exclusivity de...   
4  2022-07-11  Trending Stocks on #fintwit: \n\nSource: finap...   

                                     Processed_Tweet  Negative  Neutral  \
0  import sinc nflx may start crack password shar...     0.000    0.804   
1  googl technic look good broke day split cataly...     0.088    0.820   
2  exactli may never way weigh babatrademark valu...     0.095    0.905   
3  that negoti exclus dealthat nflx need key show...     0.000    0.804   
4  trend stock fintwit sourc finapsel tsla twtr s...     0.000    1.000   

   Positive  Compound Sentiment  
0     0.196    0.2960   Neutral  
1     0.091    0.0258   Neutral  
2     0.000   -0.2732   Neutral  
3   

In [6]:


# Load the dataset
file_path = '/content/tweet.csv'
data = pd.read_csv(file_path)

# Convert the 'Date' column to datetime format for grouping purposes
data['Date'] = pd.to_datetime(data['Date'])

# Calculating daily P_sum and P_mean
# Here, we will consider 'Compound' as the sentiment score for each tweet.
# Group by 'Date' to sum up compound scores and count tweets per day
daily_sentiment = data.groupby('Date').agg(
    P_sum=('Compound', 'sum'),        # Sum of Compound scores for each day
    twt_count=('Compound', 'size')    # Count of tweets for each day
)

# Calculate P_mean
daily_sentiment['P_mean'] = daily_sentiment['P_sum'] / daily_sentiment['twt_count']
# Save the result to a new CSV file
output_path = '/content/Olahan data Netflix.csv'
daily_sentiment.to_csv(output_path, index=False) # Changed to_excel to to_csv

print(f"The daily sentiment data has been saved to {output_path}")


The daily sentiment data has been saved to /content/Olahan data Netflix.csv
